In [4]:
from pyrosetta import *
init()
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
from scipy.spatial import *
from Bio import pairwise2
from Bio.pairwise2 import *

import os
os.system('ls *.pdb > pdb.list')
#import pyrosetta for distance calculatingand Biopython for alignment
#generate a list with all pdbs

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.25+release.d2d9f90b8cbcacfd7a1f69aefa5de610b100e8a9 2020-06-19T14:33:13] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r258 2020.25+release.d2d9f90 d2d9f90b8cbcacfd7a1f69aefa5de610b100e8a9 http://www.pyrosetta.org 2020-06-19T14:33:13
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /home/iwe2/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-187899166 seed_offset=0 real_seed=-187899166 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-187899166 RG_

0

In [9]:
def readin_files(file):
    data_list = []
    with open(file) as open_file:
        for line in open_file:
            data_list.append(line[:-1])
    return data_list
#defines the reading of files

model = readin_files('pdb.list')
assignment = readin_files('assignment.list')
volume_residues = readin_files('selected_res_vol.list')
print(model)
#read in all the necessary files and assignments

#####################################
#Define your target name from before#
#####################################
target_name = 'ADRB1'

['10_S_0001.pdb', '1_S_0001.pdb', '2_S_0001.pdb', '3_S_0001.pdb', '4_S_0001.pdb', '5_S_0001.pdb', '6_S_0001.pdb', '7_S_0001.pdb', '8_S_0001.pdb', '9_S_0001.pdb']


In [12]:
error = []
for structure in model:
    try:
        pdb_file = structure.split()
        for assign in assignment:
            assign = assign.split('\t')
            if target_name == assign[0]:
                bw_assign = "> " + assign[2]
            if target_name == assign[2]:
                bw_assign = "> " + assign[2]
            
    #Here the script tries to find the model in the assignment list, so to define bw_assign for the later BW file
        print(bw_assign)
        pose = pose_from_pdb(structure)
        fasta_model = pose.sequence()
    #The typical Pyrosetta start to read in the pdb and work with it

        file = open('all_gpcrdb_num.data', 'r')
        lines = file.readlines()
        for i in range (0, len(lines)):
            if bw_assign in lines[i]:
                position = i
        print(bw_assign)
    #The script gets the information, where in the BW list to find the needed residues

        fasta_bw = ''
        fasta = lines[position + 1]
        fasta = fasta.split('\t')
        fasta[-1] = fasta[-1][:-1]
        bw = lines[position + 2]
        bw = bw.split('\t')
        bw[-1] = bw[-1][:-1]
        for ele in fasta:
            fasta_bw = fasta_bw + ele[0]
        #From the BW all data list the sequence for BW numbering is taken, 
        #defined and a list with the names provided

        alignments = pairwise2.align.globalxx(fasta_model, fasta_bw)
        #use of Biophython lib and the module pairwise 2
        aligned_bw = alignments[0][1]
        aligned_pdb = alignments[0][0]
        #the alignment will put '-' in spaces were there are differences

        offset_bw = 0
        offset_pdb = 0
        if aligned_pdb[0] == '-':
            for residue in aligned_pdb:
                if residue != '-':
                    break
                else:
                    offset_pdb += 1
        else:
            for residue in aligned_bw:
                if residue != '-':
                    break
                else:
                    offset_bw += 1
        #need to define an offset value (bw sequence starts earlier or later)

        bw_for_alignment = []           
        #print(offset_bw, offset_pdb)


        offset_aligned_in_fasta = 0
        offset_bw_in_fasta = 0

        bw_numbering_final = []
        if offset_bw > offset_pdb: 
            for i in range(offset_bw, len(fasta_model)):
                if aligned_bw[i] == '-':
                    offset_aligned_in_fasta += 1 
                else:
                    try:
                        if aligned_pdb[i] == '-':
                            offset_bw_in_fasta += 1 
                        else:
                            bw_numbering_final.append((aligned_pdb[i], i + 1 - offset_bw_in_fasta, 
                                                       bw[i - offset_bw - offset_aligned_in_fasta]))
                    except:
                        pass
    #Here happens the magic 
    #We need two further offset values, one offset_aligned_in_fasta for the bw numbering and 
    #offset_bw_in_fasta for when in the crystal structure we have
    #another residue (complicated, but the assignment will make from ex. VASQAS BW and VASNAS determined structure 
    #-> VAS-QAS and VASN-AS)

        else:
            for i in range(offset_pdb, len(fasta_model)):
                if aligned_bw[i] == '-':
                    offset_aligned_in_fasta  += 1  
                else:
                    try:                    
                        if aligned_pdb[i] == '-':
                            offset_bw_in_fasta += 1 
                        else:
                            bw_numbering_final.append((aligned_pdb[i], (i + 1 - offset_pdb - offset_bw_in_fasta), 
                                                       bw[i - offset_aligned_in_fasta]))
                    except:
                        pass
        #prints AA, number in Rosetta and BW Numbering        


        new_file = f"{structure}.xyz_vol"
        f = open(new_file, "w")
        relevant = []
        for ele in volume_residues:
            for row in bw_numbering_final:
                if ele in row:
                    relevant.append(row)

            if relevant[-1][-1] == ele:
                pass
            else:
                relevant.append("NA")
        distance = []
        x_val = len(relevant)
        for i in range(0, (x_val * x_val)):
            distance.append(0.0)

        res_xyz = []
        for line in relevant:
            try:
        #IMPORTANT!!!! I take chain A, but its also possible that the interactions are from chain B, CHECK THIS!!!
                res = pose.residue(int(line[1]))
                res_xyz.append(res.xyz("CA"))
            except:
                res_xyz.append("NA")
        for i in range(0, len(res_xyz)):
            print(*relevant[i], res_xyz[i], sep='\t', file=f)
        f.close()
    except:
        print('\n\n\n\n' + str(structure) + '\n\n\n\n')

> ADRB1
core.import_pose.import_pose: {0} File '10_S_0001.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 83 168
core.conformation.Conformation: {0} current variant for 83 CYS
core.conformation.Conformation: {0} current variant for 168 CYS
core.conformation.Conformation: {0} current variant for 83 CYD
core.conformation.Conformation: {0} current variant for 168 CYD




10_S_0001.pdb




> ADRB1
core.import_pose.import_pose: {0} File '1_S_0001.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 83 168
core.conformation.Conformation: {0} current variant for 83 CYS
core.conformation.Conformation: {0} current variant for 168 CYS
core.conformation.Conformation: {0} current variant for 83 CYD
core.conformation.Conformation: {0} current variant for 168 CYD




1_S_0001.pdb




> ADRB1
core.import_pose.import_pose: {0} File '2_S_0001.pdb' automatically determined

In [169]:
def readin_files(file):
    data_list = []
    with open(file) as open_file:
        for line in open_file:
            data_list.append(line[:-1])
    return data_list
#defines the reading of files

model = readin_files('pdb.list')
residues = readin_files('selected_res.list')
assignment = readin_files('assignment.list')
volume_residues = readin_files('selected_res_vol.list')
print(model)
#read in all the necessary files and assignments

['4BVN.pdb']


In [15]:
#For calculating the volume of the 10 res
xyz = []
for structure in model:
    xyz.append((structure + ".xyz_vol"))

counter = 0
for ele in xyz:
    new_file = f"{ele}.vol"
    f = open(new_file, "w")
    points = []
    file = open(ele, 'r')
    try:
        for line in file:
            line = line.split()
            if "NA" not in line:
                point = line[3:]
            points.append(point)
        hull = ConvexHull(points)
        print(ele, hull.volume, file=f)
        f.close()
        counter += 1
        print(counter)
    except:
        print(ele, "NA", file=f)

1
2
3
4
5
6
7
8


In [171]:
#For calculating the distance matrix
xyz = []
for structure in model:
    xyz.append((structure + ".xyz"))

counter = 0
for ele in xyz:
    new_file = f"{ele}.matrix"
    f = open(new_file, "w")
    res_coordinates = []
    bw_number = []
    file = open(ele, 'r')
    for line in file:
        line = line.split()
        if "NA" not in line:
            x = float(line[3])
            y = float(line[4])
            z = float(line[5])
            coordinate = np.array((x,y,z))
            res_coordinates.append(coordinate)
            bw_number.append(line[2])

    for i in range(0, len(res_coordinates)):
        for o in range(i+1, len(res_coordinates)):

            distance = np.linalg.norm(np.array((res_coordinates[i]) - np.array(res_coordinates[o])))
            print(bw_number[i], 'x', bw_number[o], distance, sep='\t', file=f)

    f.close()
    counter += 1
    print(counter)

-25.571
-22.962
-17.396
-16.433
-20.153
-20.647
-24.016
-14.499
-17.04
-18.626
-19.534
-22.143
-21.007
-24.504
-16.319
-14.711
-23.316
-26.183
-23.37
-26.642
-23.179
-26.646
-30.817
-31.844
-31.317
-31.037
-33.339
-31.026
-31.326
-31.835
-29.497
-28.598
-28.466
-28.462
1
